In [1]:
import numpy as np
import spacy
from termcolor import colored
import re
confounders = {}
grammardict = {}
%run grammar_cambridge.ipynb
for key  in grammardict.keys():
    if key not in confounders.keys():
        confounders[key] = (1, [])
confounders["Proper noun"]

['about', 'above', 'across', 'after', 'against', 'along', 'alongside', 'among', 'amongst', 'around', 'as', 'at', 'before', 'behind', 'below', 'beneath', 'beside', 'besides', 'beyond', 'by', 'despite', 'down', 'during', 'except', 'for', 'from', 'in', 'inside', 'into', 'like', 'near', 'of', 'on', 'onto', 'opposite', 'over', 'round', 'since', 'to', 'towards', 'under', 'with', 'within', 'without']
['another', 'any', 'each', 'either', 'enough', 'few', 'half', 'her', 'his', 'its', 'little', 'many', 'my', 'other', 'our', 'several', 'some', 'that', 'their', 'these', 'this', 'those', 'your']
['about', 'abroad', 'absolutely', 'afterwards', 'again', 'ago', 'alike', 'almost', 'already', 'also', 'altogether', 'always', 'anyway', 'around', 'away', 'back', 'barely', 'before', 'beforehand', 'below', 'beneath', 'besides', 'beyond', 'completely', 'down', 'downwards', 'early', 'either', 'enough', 'especially', 'even', 'eventually', 'ever', 'extremely', 'fairly', 'far', 'finally', 'first', 'greatly', 'hal

(1, [])

In [2]:
import dill as pickle

In [3]:
nlp = spacy.load("en_core_web_md")

In [4]:
sorted(grammardict.keys())

['-er and -er',
 'A bit',
 'A little (downtoner)',
 'A long way',
 'A lot',
 'According to',
 'Actually',
 'Adjective + at (eg good at)',
 'Adjective after pronoun',
 'Adverb clause: after',
 'Adverb clause: although',
 'Adverb clause: as',
 'Adverb clause: because',
 'Adverb clause: before',
 'Adverb clause: during',
 'Adverb clause: even',
 'Adverb clause: finally',
 'Adverb clause: if',
 'Adverb clause: inside',
 'Adverb clause: lastly',
 'Adverb clause: now',
 'Adverb clause: once',
 'Adverb clause: since',
 'Adverb clause: supposing',
 'Adverb clause: that',
 'Adverb clause: unless',
 'Adverb clause: until',
 'Adverb clause: when',
 'Adverb clause: whenever',
 'Adverb clause: whether',
 'Adverb clause: while',
 'Adverb clause: with',
 'Adverb: about',
 'Adverb: abroad',
 'Adverb: absolutely',
 'Adverb: afterwards',
 'Adverb: again',
 'Adverb: ago',
 'Adverb: alike',
 'Adverb: almost',
 'Adverb: already',
 'Adverb: also',
 'Adverb: altogether',
 'Adverb: always',
 'Adverb: anyway',

In [5]:
for key  in grammardict.keys():
    if key not in confounders.keys():
        confounders[key] = (1, [])
confounders["Proper noun"]

(1, [])

In [6]:
# get numerical keys for the grammardict

grammar_ids = {}

for (i, idx) in enumerate(sorted(list(grammardict.keys()))):
    grammar_ids[i] = idx

In [7]:
def grammar_analyse(sentence):
    sentence = nlp(sentence)
    keys = sorted(list(grammardict.keys()))
    n = len(sentence)
    masks = []
    for i,key in enumerate(keys):
        newmask = i * grammardict[key](sentence)
        if np.sum(newmask) > 0:
            masks.append(newmask)
    mask = [[int(x[k]) for x in masks if x[k] != 0] for k in range(n)]
    display(sentence, mask)

In [8]:
def display(para, mask):
    punct_points= [-1]
    sentences = []
    masks = []
    for token in para:
        if token.text in [".", "?", "!"]: punct_points.append(token.i)
    for k,z in enumerate(punct_points[:-1]):
        sentences.append(para[z+1:punct_points[k+1]+1])
        masks.append(mask[z+1:punct_points[k+1]+1])
    for k, sentence in enumerate(sentences):
        display_sentence(sentence, masks[k])

In [9]:
a = [1,2]
a.remove(3)
a

ValueError: list.remove(x): x not in list

In [10]:
def delete_errors(mask):
    keys = sorted(list(grammardict.keys()))
    for i,z in enumerate(mask):
        names = [keys[y] for y in z]
        if z:
            for y in z:
                if len(confounders[keys[y]]) > 1:
                    for conf in confounders[keys[y]][1]:
                        if conf in names: z.remove(keys.index(conf))

    return mask

In [11]:
def resolve_confounders(mask):
    keys = sorted(list(grammardict.keys()))
    
    # remove any obvious overwrites (one is obviously bigger than the rest)
    
    for i,z in enumerate(mask):
        if z:
            ranks = []
            new = []
            for y in z:
                if keys[y] in confounders.keys():
                    ranks.append(confounders[keys[y]][0])
                else: ranks.append(1)
            top = max(ranks)
            for j,y in enumerate(z):
                if ranks[j] == top:
                    new.append(y)
            mask[i] = new
    return mask

In [12]:
def display_sentence(sentence, grammar_mask):
    colors = ['red', 'blue', 'green', 'magenta', 'cyan', 'grey','white']*10
    used = 0
    out = ""
    colordict = {}
    keys = sorted(list(grammardict.keys()))
    grammar_mask = delete_errors(grammar_mask)
    print(grammar_mask)
    grammar_mask = resolve_confounders(grammar_mask)
    print(grammar_mask)
    print("\n")
    
    for i,token in enumerate(sentence):
        if grammar_mask[i]:
            item = np.max(grammar_mask[i]) # currently look only at max grammar 
            if item != 0:
                if item in colordict.keys():
                    if not token.is_punct: out += " "
                    out += colored(token.text, colordict[item])
                else:
                    if not token.is_punct: out += " "
                    colordict[item] = colors[used]
                    used += 1
                    out += colored(token.text, colordict[item])
        else:
            if not token.is_punct: out += " "
            out += token.text
    print(out.strip())
    print("\n")
    
    for item in colordict.keys():
        print(colored(keys[item], colordict[item]))
        
    print("\n")

In [14]:
confounders["Adverb clause: before"]

(1, [])

In [15]:
sentences = "The plane was late and detectives were waiting at the airport all morning. They were expecting a valuable parcel of diamonds from South Africa. A few hours earlier, someone had told the police that thieves would try to steal the diamonds. When the plane arrived, some of the detectives were waiting inside the main building while others were waiting on the airfield."

In [18]:
sentence = input("> ")
grammar_analyse(sentence)

> It had never been a better time to try to eat some cheese.
[[], [791, 845, 851], [88], [788, 845, 851], [], [171, 1884], [], [], [787], [787], [787], [191], [], []]
[[], [851], [88], [851], [], [1884], [], [], [787], [787], [787], [191], [], []]


It had never been a better time to try to eat some cheese.


Past pefect
Adverb: never
better
Infinitive (intention)
Determiner: some




In [ ]:
sentence = nlp("He did the job.")
for token in sentence:
    print(token.text, token.lemma_.lower(), token.pos_, token.head.lemma_, list(token.subtree))

In [ ]:
keys = sorted(grammardict.keys())
keys[674]